In [1]:
import numpy as np
import pandas as pd
import pickle
import os
import re
import jieba
import random
import math

In [2]:
def word2idx(embedding):
    wordidx = {}
    for i in range(len(embedding)):
        word=embedding['name'][i]
        wordidx[word] = len(wordidx)
    return(wordidx)           
            
def sen2idx(sen, wordidx):
    idxword = []
    for s in sen:
        try:
            idx = wordidx[s]
            idxword.append(idx)
        except:
            continue
    return(idxword)

In [3]:
os.chdir(r'/Replication_TrainingDatasetCensorship') # set working directory

In [4]:
wk = pd.read_csv('output/2.dropna_wiki_clean.csv')
wk.rename(columns={'Unnamed: 0': 'name'}, inplace=True)
wk = wk.dropna()
wk.drop_duplicates(subset=['name'], inplace=True)
wk = wk.reset_index(drop=True)
wk.head()

,name,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301
0,，,0.150785,0.120591,-0.220204,-0.044533,0.374161,0.044214,-0.392151,0.035214,0.300220,...,-0.187215,-0.150791,0.339811,0.069382,-0.323648,0.108729,0.065358,-0.532687,-0.066914,-0.100728
1,的,-0.251355,0.234742,-0.169728,0.026955,0.258850,0.028063,-0.498176,0.076507,0.151926,...,0.206175,-0.320072,0.206702,0.297621,-0.255238,0.076537,-0.021104,-0.527221,-0.229531,0.180781
2,。,0.207820,0.141425,-0.337281,0.158955,0.232709,0.008633,-0.401047,-0.088699,0.113330,...,0.108845,-0.074981,0.104715,-0.035879,-0.365303,-0.142196,0.015788,-0.555588,0.047561,-0.070897
3,、,0.020587,-0.472420,-0.441270,0.650659,0.406180,-0.197017,-0.230998,-0.362920,-0.372085,...,0.111319,-0.080049,0.768271,0.100696,-0.348324,-0.236327,0.031901,-0.870347,0.510523,0.065956
4,和,0.009246,0.187802,0.066590,-0.095002,0.193442,0.067781,-0.194179,-0.203283,-0.278398,...,0.300195,-0.004085,0.388441,-0.042360,-0.064346,-0.161472,0.136743,-0.496433,0.005538,0.038096


In [5]:
wknp = wk.loc[:,'V2':'V301'].values
np.savez('TextCNN/data/wknp.npz', wknp)

In [6]:
wk_wordidx = word2idx(wk)
with open('TextCNN/data/vocab_wk.pkl', 'wb') as f:
    pickle.dump(wk_wordidx, f)

In [7]:
bd = pd.read_csv('output/2.dropna_baidubaike_clean.csv')
bd.rename(columns={'Unnamed: 0': 'name'}, inplace=True)
bd = bd.dropna()
bd.drop_duplicates(subset=['name'], inplace=True)
bd = bd.reset_index(drop=True)
bdnp = bd.loc[:,'V2':'V301'].values
np.savez('TextCNN/data/bdnp.npz', bdnp)
bd_wordidx = word2idx(bd)
with open('TextCNN/data/vocab_bd.pkl', 'wb') as f:
    pickle.dump(bd_wordidx, f)

In [9]:
rm = pd.read_csv('output/2.dropna_renmin_clean.csv')
rm.rename(columns={'Unnamed: 0': 'name'}, inplace=True)
rm = rm.dropna()
rm.drop_duplicates(subset=['name'], inplace=True)
rm = rm.reset_index(drop=True)
rmnp = rm.loc[:,'V2':'V301'].values
np.savez('TextCNN/data/rmnp.npz', rmnp)
rm_wordidx = word2idx(rm)
with open('TextCNN/data/vocab_rm.pkl', 'wb') as f:
    pickle.dump(rm_wordidx, f)

In [13]:
news = pd.read_csv('auxiliary/Political_news_titles.csv')
news = news[news['label'].isnull()==False]
punct = ['......', '，', '--', '：',  '丨', '「', '」', '、', '——',  '-',  '_', '...', '·', '。',
        '【', '】', '《', '》', '/', '(图)', '(组图)', '[图]', '‧', '.', ':', ',', '（下）', '（上）', '（图）', '（组图）', ' ',
        '丨', '|', '(', ')', '（', '）', '[', ']', "'", '’', '‘', '“', '”', '%', '?', '!']
for i in range(len(news)):
    text = news.iloc[i,0]
    for p in punct:
      text = text.replace(p, "")
    text = re.sub('[a-zA-Z]','',text)
    text = re.sub("\d+", "", text)
    news.iloc[i,0] = text
news = news[news['label']!=0]
news.drop_duplicates(subset=['sentence'], inplace=True)
news = news.dropna(how='any',axis=0) 
news = news.reset_index(drop=True)
news["label"] = news["label"].astype(int)

In [14]:
for i in range(len(news)):
    if news.iloc[i, 1]==-1:
        news.iloc[i, 1] = 0
news.head()

,sentence,label
0,达里奥资本市场已不再自由贫富差距达年代以来最严重,0
1,新闻自由日联合国秘书长强调记者为新冠大流行错误信息提供,0
2,一组老照片自由女神像是如何诞生的,1
3,在互联网自由日抗议苹果公司同中共合作审查网络,0
4,香港居民权利自由的坚实保障,1


In [15]:
train = pd.read_csv('auxiliary/TNEWS_subset.csv')
train = train[train['label'].isnull()==False]
for i in range(len(train)):
    text = train.iloc[i,0]
    for p in punct:
      text = text.replace(p, "")
    text = re.sub('[a-zA-Z]','',text)
    text = re.sub("\d+", "", text)
    train.iloc[i,0] = text
train = train[train['label']!=0]
train.drop_duplicates(subset=['sentence'], inplace=True)
train = train.dropna(how='any',axis=0) 
train = train.reset_index(drop=True)

In [16]:
for i in range(len(train)):
    if train.iloc[i, 1]==-1:
        train.iloc[i, 1] = 0
train.head()

,sentence,label
0,年这台轿车最值得期待最后那台要火,1
1,世界杯组解盘高卢雄鸡一枝独秀倒计时天,1
2,这一金牌大项或被踢出奥运会部分裁判全部被禁赛名将含冤落泪,0
3,这个字险啊险险得超乎你的想象！,0
4,叙政府军清点战利品时发现了些产自中国的东西足以震惊世界,1


In [17]:
jieba.load_userdict("auxiliary/Dict.txt")

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/8g/74cy_pd90b94x2b6lxyf92v40000gn/T/jieba.cache
Loading model cost 0.545 seconds.
Prefix dict has been built successfully.


In [18]:
for i in range(len(news)):
    c = jieba.lcut(news.iloc[i, 0])
    news.iloc[i, 0] = ' '.join(c)
news.head()

,sentence,label
0,达里奥 资本 市场 已 不再 自由 贫富差距 达 年代 以来 最 严重,0
1,新闻自由 日 联合国 秘书长 强调 记者 为 新冠大 流行 错误信息 提供,0
2,一组 老照片 自由女神 像是 如何 诞生 的,1
3,在 互联网 自由 日 抗议 苹果公司 同 中共 合作 审查 网络,0
4,香港 居民 权利 自由 的 坚实 保障,1


In [19]:
for i in range(len(train)):
    c = jieba.lcut(train.iloc[i, 0])
    train.iloc[i, 0] = ' '.join(c)
train.head()

,sentence,label
0,年 这台 轿车 最 值得 期待 最后 那台 要 火,1
1,世界杯 组解盘 高卢 雄鸡 一枝独秀 倒计时 天,1
2,这一 金牌 大项 或 被 踢 出 奥运会 部分 裁判 全部 被 禁赛 名将 含冤 落泪,0
3,这个 字险 啊 险险 得 超乎 你 的 想象 ！,0
4,叙 政府军 清点 战利品 时 发现 了 些 产自 中国 的 东西 足以 震惊 世界,1


In [20]:
random.seed(92092)
dev_idx = random.sample(range(len(train)), math.floor(len(train)*0.2))
len(dev_idx)

484

In [21]:
train_idx = [j for j in range(len(train)) if j not in dev_idx] 
len(train_idx)

1936

In [22]:
trainset = train.iloc[train_idx]
dev = train.iloc[dev_idx]

In [23]:
news = news.to_dict('records')
train = trainset.to_dict('records')
dev = dev.to_dict('records')

In [24]:
with open('TextCNN/data/train.txt', 'w') as fp:
    for t in train:
        fp.write(t['sentence'] + '__' + str(t['label']) + '\n')

In [25]:
with open('TextCNN/data/dev.txt', 'w') as fp:
    for t in dev:
        fp.write(t['sentence'] + '__' + str(t['label']) + '\n')

In [26]:
with open('TextCNN/data/test.txt', 'w') as fp:
    for t in news:
        fp.write(t['sentence'] + '__' + str(t['label']) + '\n')